#패키지 불러오기

In [8]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
% matplotlib inline
from datetime import datetime
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.utils import class_weight
from sklearn.neighbors import KNeighborsClassifier

#데이터 불러오기 및 전처리

In [9]:
data_train = pd.read_csv('/content/train.csv')

In [10]:
data_train = data_train.fillna('unknown')

X = data_train.drop(columns=['index', 'credit'])
y = data_train['credit']

X_encoded = pd.get_dummies(X, columns=['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type'])

In [11]:
X_encoded

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_unknown
0,0,202500.0,-13899,-4709,1,0,0,0,2.0,-6.0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,247500.0,-11380,-1540,1,0,0,1,3.0,-5.0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,450000.0,-19087,-4434,1,0,1,0,2.0,-22.0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,202500.0,-15088,-2092,1,0,1,0,2.0,-37.0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,157500.0,-15037,-2105,1,0,0,0,2.0,-26.0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,2,225000.0,-12079,-1984,1,0,0,0,4.0,-2.0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26453,1,180000.0,-15291,-2475,1,0,0,0,2.0,-47.0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
26454,0,292500.0,-10082,-2015,1,0,0,0,2.0,-25.0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26455,0,171000.0,-10145,-107,1,0,0,0,1.0,-59.0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [12]:
x_train, x_valid, y_train, y_valid = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

In [13]:
x_train

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_unknown
23445,2,270000.0,-13281,-312,1,0,1,0,3.0,-13.0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
13278,0,220500.0,-15130,-170,1,0,0,0,2.0,-22.0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9303,2,180000.0,-12161,-127,1,1,0,0,4.0,-35.0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7794,2,360000.0,-10685,-1863,1,1,0,0,4.0,-23.0,1,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
10507,0,180000.0,-13774,-2684,1,0,0,0,1.0,-48.0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,3,135000.0,-14838,-1539,1,0,0,0,5.0,-47.0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5390,1,69750.0,-10167,-829,1,1,1,0,3.0,-46.0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
860,0,58500.0,-20657,365243,1,0,0,0,2.0,-12.0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
15795,1,112500.0,-9811,-864,1,0,0,0,3.0,-35.0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


test set 전처리

In [14]:
data_test = pd.read_csv('/content/test.csv')

In [15]:
data_test = data_test.fillna('unknown')

x_test = data_test.drop(columns=['index'])

x_test = pd.get_dummies(x_test, columns=['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type'])

In [16]:
x_test

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_unknown
0,0,112500.0,-21990,365243,1,0,1,0,2.0,-60.0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,135000.0,-18964,-8671,1,0,1,0,2.0,-36.0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,69372.0,-15887,-217,1,1,1,0,2.0,-40.0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,112500.0,-19270,-2531,1,1,0,0,2.0,-41.0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,225000.0,-17822,-9385,1,1,0,0,2.0,-8.0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,202500.0,-18593,-5434,1,1,1,0,2.0,-19.0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9996,0,202500.0,-10886,-1315,1,1,0,0,2.0,-34.0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9997,0,292500.0,-21016,-14018,1,0,0,0,2.0,-55.0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
9998,0,180000.0,-16541,-1085,1,0,1,0,2.0,-33.0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


# classification

In [17]:
def submission(classifier, name):
  curr_time = datetime.now().strftime('%y%m%d%H%M%S')
  X_raw = pd.read_csv('/content/test.csv', encoding='utf-8')
  l_idx = X_raw['index']
  submission = pd.DataFrame(np.insert(classifier.predict_proba(x_test), 0, l_idx, axis=1), columns=['index', 0, 1, 2])
  submission['index'] = submission['index'].astype('int')
  submission.to_csv(f'submission_{curr_time}_{name}.csv', index=False)

ADABoost

In [44]:
from sklearn.ensemble import AdaBoostClassifier

In [45]:
clf_ADA = AdaBoostClassifier(algorithm= 'SAMME.R', learning_rate = 1, n_estimators = 10)
clf_ADA.fit(x_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
                   n_estimators=10, random_state=None)

In [46]:
y_pred_adab = clf_ADA.predict_proba(x_valid)

In [47]:
log_loss(y_true=y_valid, y_pred=y_pred_adab)

1.031530544512119

SVM

In [22]:
clf_svc = SVC(kernel='sigmoid',
              probability = True,
              random_state=42)
clf_svc.fit(x_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='sigmoid',
    max_iter=-1, probability=True, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [23]:
y_pred_svc = clf_svc.predict_proba(x_valid)

In [24]:
log_loss(y_true=y_valid, y_pred=y_pred_svc)

0.8859904322085493

XGB

In [25]:
from xgboost import XGBClassifier

In [26]:
clf_xgb = XGBClassifier(learning_rate = 0.01, max_depth = 10)

In [27]:
clf_xgb.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [28]:
y_pred_xgb = clf_xgb.predict_proba(x_valid)
log_loss(y_true=y_valid, y_pred=y_pred_xgb)

0.7457110987318355

random forest


In [30]:
from sklearn.ensemble import RandomForestClassifier

In [31]:
clf_rf = RandomForestClassifier(criterion= 'entropy', max_depth= 20, max_features= 'sqrt', n_estimators= 30)
clf_rf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=20, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [32]:
y_pred_dt = clf_rf.predict_proba(x_valid)

In [33]:
log_loss(y_valid, y_pred_dt)

0.754250910420116

lgbm

In [35]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

In [36]:
lgbm = LGBMClassifier(learning_rate=0.01)

In [37]:
lgbm.fit(x_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.01, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [38]:
y_pred_lgbm = lgbm.predict_proba(x_valid)

In [40]:
y_pred_lgbm

array([[0.10090909, 0.24302749, 0.65606343],
       [0.11641049, 0.26917048, 0.61441903],
       [0.11877629, 0.22426417, 0.65695954],
       ...,
       [0.10856179, 0.22390867, 0.66752954],
       [0.13345826, 0.25332161, 0.61322013],
       [0.10938084, 0.19087396, 0.6997452 ]])

In [42]:
log_loss(y_valid, y_pred_lgbm)

0.8229771024790687

ENSEMBLE

In [43]:
from sklearn.ensemble import VotingClassifier

In [58]:
ensemble = VotingClassifier(
    estimators=[('ada', clf_ADA), ('svc', clf_svc), ('xgb', clf_xgb), ('rf', clf_rf), ('lgbm', lgbm)], weights=[1,1,15,1,1], voting='soft'    
)

In [59]:
ensemble.fit(x_train, y_train)

VotingClassifier(estimators=[('ada',
                              AdaBoostClassifier(algorithm='SAMME.R',
                                                 base_estimator=None,
                                                 learning_rate=1,
                                                 n_estimators=10,
                                                 random_state=None)),
                             ('svc',
                              SVC(C=1.0, break_ties=False, cache_size=200,
                                  class_weight=None, coef0=0.0,
                                  decision_function_shape='ovr', degree=3,
                                  gamma='scale', kernel='sigmoid', max_iter=-1,
                                  probability=True, random_state=42,
                                  shrinking=T...
                                             importance_type='split',
                                             learning_rate=0.01, max_depth=-1,
                       

In [60]:
y_pred_ensemble = ensemble.predict_proba(x_valid)

In [61]:
log_loss(y_valid, y_pred_ensemble)

0.7496518655490357

In [62]:
submission(ensemble, 'ensemble')